# EOCanvas Batch Processing with C2RCC on SNAP

## Setup

In [1]:
from eocanvas import API, Credentials
from eocanvas.api import Input, Config, ConfigOption
from eocanvas.processes import SnapProcess
from eocanvas.snap.graph import Graph

In [2]:
c = Credentials.load()

In [3]:
from hda import Client

In [4]:
c = Client()

In [5]:
c.metadata("EO:ESA:DAT:SENTINEL-2")

{'type': 'object',
 'title': 'Querable',
 'properties': {'dataset_id': {'title': 'dataset_id',
   'type': 'string',
   'oneOf': [{'const': 'EO:ESA:DAT:SENTINEL-2',
     'title': 'EO:ESA:DAT:SENTINEL-2',
     'group': None}]},
  'bbox': {'title': 'Bbox',
   'type': 'array',
   'minItems': 4,
   'maxItems': 4,
   'items': [{'type': 'number', 'maximum': 180, 'minimum': -180},
    {'type': 'number', 'maximum': 90, 'minimum': -90},
    {'type': 'number', 'maximum': 180, 'minimum': -180},
    {'type': 'number', 'maximum': 90, 'minimum': -90}]},
  'productIdentifier': {'title': 'Product Identifier',
   'type': 'string',
   'pattern': '^[a-zA-Z0-9]+$'},
  'productType': {'title': 'Product Type',
   'type': 'string',
   'oneOf': [{'const': 'S2MSI1C', 'title': 'S2MSI1C', 'group': None},
    {'const': 'S2MSI2A', 'title': 'S2MSI2A', 'group': None},
    {'const': 'AUX_GNSSRD', 'title': 'AUX_GNSSRD', 'group': None},
    {'const': 'AUX_PROQUA', 'title': 'AUX_PROQUA', 'group': None},
    {'const': 'AU

Default query template.

In [6]:
q = {
    "dataset_id": "EO:ESA:DAT:SENTINEL-2",
    "startdate": "",
    "enddate": "",
    "processingLevel": "S2MSI1C",
    "tileId": ""
}

Load AOI polygons.

In [7]:
import json

In [8]:
with open('AOIs.txt', 'r') as fp:
    poly_dict = json.load(fp)

In [9]:
poly_dict

{'2': {'geoRegion': 'POLYGON ((-5.22444257 49.9901843, -4.88225479 49.9901843, -4.88225479 50.29003327, -5.22444257 50.29003327, -5.22444257 49.9901843))',
  'tileId': '30UUA'},
 '3': {'geoRegion': 'POLYGON ((-4.81917128 50.19269224, -4.59558622 50.19269224, -4.59558622 50.40467681, -4.81917128 50.40467681, -4.81917128 50.19269224))',
  'tileId': '30UUA'},
 '4': {'geoRegion': 'POLYGON ((-4.32949749 50.26538565, -4.02885352 50.26538565, -4.02885352 50.5075424, -4.32949749 50.5075424, -4.32949749 50.26538565))',
  'tileId': '30UVA'},
 '6': {'geoRegion': 'POLYGON ((-4.02836874 50.14143345, -3.59050258 50.14143345, -3.59050258 50.34954708, -4.02836874 50.34954708, -4.02836874 50.14143345))',
  'tileId': '30UVA'},
 '9': {'geoRegion': 'POLYGON ((-3.70217003 50.32171356, -3.4070158 50.32171356, -3.4070158 50.54736093, -3.70217003 50.54736093, -3.70217003 50.32171356))',
  'tileId': '30UVA'},
 '11': {'geoRegion': 'POLYGON ((-4.31178593 50.97017888, -3.95799343 50.97017888, -3.95799343 51.30272

Group by tile.

In [10]:
tile_polys = {}
for k, v in poly_dict.items():
    tileId = v['tileId']
    geo_region = v['geoRegion']

    if tileId not in tile_polys:
        tile_polys[tileId] = {}
    tile_polys[tileId][k] = geo_region

In [11]:
tile_polys

{'30UUA': {'2': 'POLYGON ((-5.22444257 49.9901843, -4.88225479 49.9901843, -4.88225479 50.29003327, -5.22444257 50.29003327, -5.22444257 49.9901843))',
  '3': 'POLYGON ((-4.81917128 50.19269224, -4.59558622 50.19269224, -4.59558622 50.40467681, -4.81917128 50.40467681, -4.81917128 50.19269224))',
  '20': 'POLYGON ((-5.7897289 49.9734232, -5.3357743 49.9734232, -5.3357743 50.27582106, -5.7897289 50.27582106, -5.7897289 49.9734232))'},
 '30UVA': {'4': 'POLYGON ((-4.32949749 50.26538565, -4.02885352 50.26538565, -4.02885352 50.5075424, -4.32949749 50.5075424, -4.32949749 50.26538565))',
  '6': 'POLYGON ((-4.02836874 50.14143345, -3.59050258 50.14143345, -3.59050258 50.34954708, -4.02836874 50.34954708, -4.02836874 50.14143345))',
  '9': 'POLYGON ((-3.70217003 50.32171356, -3.4070158 50.32171356, -3.4070158 50.54736093, -3.70217003 50.54736093, -3.70217003 50.32171356))'},
 '30UVB': {'11': 'POLYGON ((-4.31178593 50.97017888, -3.95799343 50.97017888, -3.95799343 51.3027289, -4.31178593 51.3

Load query dates for each tile.

In [12]:
import csv

In [13]:
tile_dates = {}

In [14]:
with open("query_dates.csv", "r") as fp:
    reader = csv.reader(fp, delimiter=",")
    next(reader)
    for tileId, startdate, enddate in reader:
        tile_dates[tileId] = {'startdate': startdate, 'enddate': enddate}

In [15]:
tile_dates

{'30UUA': {'startdate': '2020-06-23T00:00:00.000Z',
  'enddate': '2020-06-24T00:00:00.000Z'},
 '30UVA': {'startdate': '2020-06-23T00:00:00.000Z',
  'enddate': '2020-06-24T00:00:00.000Z'},
 '30UVB': {'startdate': '2020-06-23T00:00:00.000Z',
  'enddate': '2020-06-24T00:00:00.000Z'},
 '30UWB': {'startdate': '2020-06-25T00:00:00.000Z',
  'enddate': '2020-06-26T00:00:00.000Z'},
 '30UXB': {'startdate': '2020-07-30T00:00:00.000Z',
  'enddate': '2020-07-31T00:00:00.000Z'},
 '30UWC': {'startdate': '2020-06-25T00:00:00.000Z',
  'enddate': '2020-06-26T00:00:00.000Z'},
 '29UPR': {'startdate': '2020-06-01T00:00:00.000Z',
  'enddate': '2020-06-02T00:00:00.000Z'},
 '30UUC': {'startdate': '2020-06-01T00:00:00.000Z',
  'enddate': '2020-06-02T00:00:00.000Z'},
 '30UVC': {'startdate': '2020-06-23T00:00:00.000Z',
  'enddate': '2020-06-24T00:00:00.000Z'},
 '31UCS': {'startdate': '2023-07-07T00:00:00.000Z',
  'enddate': '2023-07-08T00:00:00.000Z'},
 '31UCT': {'startdate': '2023-07-07T00:00:00.000Z',
  'endda

## Process workflow

In [16]:
with open("s2_c2rcc.xml", "r") as fp:
    graph_file = fp.read()

In [17]:
# common to all processes
config = Config(key="img1", options=ConfigOption(uncompress=False, sub_path=""))

In [18]:
tiles_to_process = ['30UUC', '30UVD', '30UXB']

In [19]:
polys_to_process = ['18', '27', '14']

In [20]:
import os

In [21]:
for tileId in tiles_to_process:
    # Query parameters depend on tile
    q['tileId'] = tileId
    query_dates = tile_dates[tileId]
    q['startdate'] = query_dates['startdate']
    q['enddate'] = query_dates['enddate']
    r = c.search(q)
    # if done correctly, there should only be one or two results. Choose first.
    if not r:
        print(f"[WARNING] Search for query containing tileId {tileId} failed. Found none.")
        continue
    url = r.get_download_urls()[0]
    inputs = Input(key="img1", url=url)

    for poly_id, poly in tile_polys[tileId].items():
        if poly_id not in polys_to_process:
            continue
        modified_xml = graph_file.replace("$polygon", poly)
        graph = Graph.from_text(modified_xml)
        process = SnapProcess(snap_graph=graph, eo_config=config, eo_input=inputs)
        process.prepare_inputs()
        try:
            process.run(download_dir=f"result-{tileId}")  # save unordered in result-{tileId} directory
        except:
            print(f"[ERROR] Failed to process {tileId} poly {poly}")
            continue

Job: 8287116b-5c53-590e-8cfc-8a1423d4e6e9 - Status: accepted at 2025-02-27T17:11:06.978903
Job: 8287116b-5c53-590e-8cfc-8a1423d4e6e9 - Status: running at 2025-02-27T17:11:17.086415
Job: 8287116b-5c53-590e-8cfc-8a1423d4e6e9 - Status: running at 2025-02-27T17:11:28.190286
Job: 8287116b-5c53-590e-8cfc-8a1423d4e6e9 - Status: running at 2025-02-27T17:11:40.447185
Job: 8287116b-5c53-590e-8cfc-8a1423d4e6e9 - Status: running at 2025-02-27T17:11:53.801051
Job: 8287116b-5c53-590e-8cfc-8a1423d4e6e9 - Status: running at 2025-02-27T17:12:08.502726
Job: 8287116b-5c53-590e-8cfc-8a1423d4e6e9 - Status: running at 2025-02-27T17:12:24.892741
Job: 8287116b-5c53-590e-8cfc-8a1423d4e6e9 - Status: running at 2025-02-27T17:12:42.924815
Job: 8287116b-5c53-590e-8cfc-8a1423d4e6e9 - Status: running at 2025-02-27T17:13:02.510530
Job: 8287116b-5c53-590e-8cfc-8a1423d4e6e9 - Status: running at 2025-02-27T17:13:24.250024
Job: 8287116b-5c53-590e-8cfc-8a1423d4e6e9 - Status: running at 2025-02-27T17:13:47.960853
Job: 8287